# 20231202 - Scrambled

## Task - Initial task
Over natten har det vært store utfordringer knyttet til en av maskinene i verkstedet.  
En serie feilproduserte leker har kommet på rullende bånd.  
Vi prøver å finne ut hva som har skjedd. Graver du ned i det her?   
Mellomleder

**Vedlegg:**  
<img src="./data/20231202-oedelagte_leker.png" alt="evidence image" width="400" />

## Task - additional information
Tror det var noe skitt på linsa da det forrige bildet ble tatt, så jeg har tatt et nytt et.  
Har lagt med som vedlegg.  
Tastefinger

**Vedlegg:**  
<img src="./data/20231202-oedelagte_leker_fix.png" alt="evidence image 2" width="500" />

## Answer
I started by finding a piece of software which would allow me to solve the cube.  
No software was readily available, but after finding the rubiks cube solver from pglass, https://github.com/pglass/cube, I disected the "cube.py", and added classes from the "maths" import.  Then significantly shortened and remove bloat-functions.

One problem is the format of the cube, forces you to rotate the original image in order to map it correctly to the faces. Doing so results in a rotated view of the solution, making it harder to "find" the solution in the text as you must read from left to right, from the bottom up.  The great thing about this scaled down version is that the values in the cube can be any character value, as long as you provide it with the sequence of operations to solve the cube.  Which is why I chose this software.

**THIS TOOK ME HOURS TO GRASP.**  Implementing the solution and finding the correct steps to solve the cube took 30min.

So, in order to get the sequence of operations to solve the cube, I Googled and found a Rubiks cube solver on https://rubikscu.be.  Which allows you to map the colours of the cube and run an autosolver on it, which then also prints the standard text notation needed by the software.

<img src="./data/20231202-rubiks_cube_solver.png" alt="drawing" width="400" />

**FLAG:** `PST{LØSTE_DU_DENNE_SOM_PUSLESPILL_ELLER_KUBE?:)}`

## Solution details
So adding the things described above together I started with the following mappings of the cube.  
**Colour and value mapped cubes, flattened**
```
   bbw             {EO
   ogy             LS)
   wgr             ELW
yggrygyrogwr   :SMT?P}BLENR
grrwwoyorbyr   _OSDNEKEGLEY
boyowrwgobby   __URSP_UIUS_
   gob             PLØ
   bbw             DTE
   oyw             LB_
```

Then I got the following sequence from the online solver, which yielded unscrambled cube faces:   
`U B Ui B Li B R Li B B U F B B Ui B B R R F F B B D D B B D`

Running the text-cube through the solver, I got the following solution  
**Value mapped cube**
```
    PLP
    _SS
    MUE
WYT EEO LLR U?}
BOS DNS LEE KE)
RGP _N_ I_L _B:
    ØEU
    LTD
    {S_
```

Which yielded the **FLAG:** `PST{LØSTE_DU_DENNE_SOM_PUSLESPILL_ELLER_KUBE?:)}`

In [ ]:
import string
class Point:
    def __init__(self, x, y=None, z=None):
        """Construct a Point from an (x, y, z) tuple or an iterable"""
        try:
            # convert from an iterable
            ii = iter(x)
            self.x = next(ii)
            self.y = next(ii)
            self.z = next(ii)
        except TypeError:
            # not iterable
            self.x = x
            self.y = y
            self.z = z
        if any(val is None for val in self):
            raise ValueError(f"Point does not allow None values: {self}")

    def __add__(self, other):
        return Point(self.x + other.x, self.y + other.y, self.z + other.z)

    def __sub__(self, other):
        return Point(self.x - other.x, self.y - other.y, self.z - other.z)

    def dot(self, other):
        """Return the dot product"""
        return self.x * other.x + self.y * other.y + self.z * other.z

    def __getitem__(self, item):
        if item == 0:
            return self.x
        elif item == 1:
            return self.y
        elif item == 2:
            return self.z
        raise IndexError("Point index out of range")
    
    def count(self, val):
        return int(self.x == val) + int(self.y == val) + int(self.z == val)

class Matrix:
    def __init__(self, *args):
        if len(args) == 9:
            self.vals = list(args)
        elif len(args) == 3:
            try:
                self.vals = [x for y in args for x in y]
            except Exception:
                self.vals = []
        else:
            self.__init__(*args[0])

        if len(self.vals) != 9:
            raise ValueError(f"Matrix requires 9 items, got {args}")

    def __mul__(self, other):
        """Do Matrix-Matrix or Matrix-Point multiplication."""
        if isinstance(other, Point):
            return Point(other.dot(Point(row)) for row in self.rows())
        elif isinstance(other, Matrix):
            return Matrix(Point(row).dot(Point(col)) for row in self.rows() for col in other.cols())

    def rows(self):
        yield self.vals[0:3]
        yield self.vals[3:6]
        yield self.vals[6:9]


RIGHT = X_AXIS = Point(1, 0, 0)
LEFT           = Point(-1, 0, 0)
UP    = Y_AXIS = Point(0, 1, 0)
DOWN           = Point(0, -1, 0)
FRONT = Z_AXIS = Point(0, 0, 1)
BACK           = Point(0, 0, -1)

FACE = 'face'
EDGE = 'edge'
CORNER = 'corner'

# 90 degree rotations in the XY plane. CW is clockwise, CC is counter-clockwise.
ROT_XY_CW = Matrix(0, 1, 0, -1, 0, 0, 0, 0, 1)
ROT_XY_CC = Matrix(0, -1, 0, 1, 0, 0, 0, 0, 1)
# 90 degree rotations in the XZ plane (around the y-axis when viewed pointing toward you).
ROT_XZ_CW = Matrix(0, 0, -1, 0, 1, 0, 1, 0, 0)
ROT_XZ_CC = Matrix(0, 0, 1, 0, 1, 0, -1, 0, 0)
# 90 degree rotations in the YZ plane (around the x-axis when viewed pointing toward you).
ROT_YZ_CW = Matrix(1, 0, 0, 0, 0, 1, 0, -1, 0)
ROT_YZ_CC = Matrix(1, 0, 0, 0, 0, -1, 0, 1, 0)

class Piece:
    def __init__(self, pos, colors):
        assert all(type(x) == int and x in (-1, 0, 1) for x in pos)
        assert len(colors) == 3
        self.pos = pos
        self.colors = list(colors)
        self._set_piece_type()


    def _set_piece_type(self):
        if self.colors.count(None) == 2:
            self.type = FACE
        elif self.colors.count(None) == 1:
            self.type = EDGE
        elif self.colors.count(None) == 0:
            self.type = CORNER
        else:
            raise ValueError(f"Must have 1, 2, or 3 colors - given colors={self.colors}")

    def rotate(self, matrix):
        before = self.pos
        self.pos = matrix * self.pos
        rot = self.pos - before
        if not any(rot):
            return  # no change occurred
        if rot.count(0) == 2:
            rot += matrix * rot
        assert rot.count(0) == 1, (
            f"There is a bug in the Piece.rotate() method!"
            f"\nbefore: {before}"
            f"\nself.pos: {self.pos}"
            f"\nrot: {rot}"
        )
        i, j = (i for i, x in enumerate(rot) if x != 0)
        self.colors[i], self.colors[j] = self.colors[j], self.colors[i]

class Cube:
    def __init__(self, cube_str):
        cube_str = "".join(x for x in cube_str if x not in string.whitespace)
        assert len(cube_str) == 54
        self.faces = (
            Piece(pos=RIGHT, colors=(cube_str[28], None, None)),
            Piece(pos=LEFT,  colors=(cube_str[22], None, None)),
            Piece(pos=UP,    colors=(None, cube_str[4],  None)),
            Piece(pos=DOWN,  colors=(None, cube_str[49], None)),
            Piece(pos=FRONT, colors=(None, None, cube_str[25])),
            Piece(pos=BACK,  colors=(None, None, cube_str[31])))
        self.edges = (
            Piece(pos=RIGHT + UP,    colors=(cube_str[16], cube_str[5], None)),
            Piece(pos=RIGHT + DOWN,  colors=(cube_str[40], cube_str[50], None)),
            Piece(pos=RIGHT + FRONT, colors=(cube_str[27], None, cube_str[26])),
            Piece(pos=RIGHT + BACK,  colors=(cube_str[29], None, cube_str[30])),
            Piece(pos=LEFT + UP,     colors=(cube_str[10], cube_str[3], None)),
            Piece(pos=LEFT + DOWN,   colors=(cube_str[34], cube_str[48], None)),
            Piece(pos=LEFT + FRONT,  colors=(cube_str[23], None, cube_str[24])),
            Piece(pos=LEFT + BACK,   colors=(cube_str[21], None, cube_str[32])),
            Piece(pos=UP + FRONT,    colors=(None, cube_str[7], cube_str[13])),
            Piece(pos=UP + BACK,     colors=(None, cube_str[1], cube_str[19])),
            Piece(pos=DOWN + FRONT,  colors=(None, cube_str[46], cube_str[37])),
            Piece(pos=DOWN + BACK,   colors=(None, cube_str[52], cube_str[43])),
        )
        self.corners = (
            Piece(pos=RIGHT + UP + FRONT,   colors=(cube_str[15], cube_str[8], cube_str[14])),
            Piece(pos=RIGHT + UP + BACK,    colors=(cube_str[17], cube_str[2], cube_str[18])),
            Piece(pos=RIGHT + DOWN + FRONT, colors=(cube_str[39], cube_str[47], cube_str[38])),
            Piece(pos=RIGHT + DOWN + BACK,  colors=(cube_str[41], cube_str[53], cube_str[42])),
            Piece(pos=LEFT + UP + FRONT,    colors=(cube_str[11], cube_str[6], cube_str[12])),
            Piece(pos=LEFT + UP + BACK,     colors=(cube_str[9], cube_str[0], cube_str[20])),
            Piece(pos=LEFT + DOWN + FRONT,  colors=(cube_str[35], cube_str[45], cube_str[36])),
            Piece(pos=LEFT + DOWN + BACK,   colors=(cube_str[33], cube_str[51], cube_str[44])),
        )
        self.pieces = self.faces + self.edges + self.corners

    def _face(self, axis):
        assert axis.count(0) == 2
        return [p for p in self.pieces if p.pos.dot(axis) > 0]

    def _slice(self, plane):
        assert plane.count(0) == 1
        i = next((i for i, x in enumerate(plane) if x == 0))
        return [p for p in self.pieces if p.pos[i] == 0]

    def _rotate_face(self, face, matrix):
        self._rotate_pieces(self._face(face), matrix)

    def _rotate_slice(self, plane, matrix):
        self._rotate_pieces(self._slice(plane), matrix)

    def _rotate_pieces(self, pieces, matrix):
        for piece in pieces:
            piece.rotate(matrix)

    # Rubik's Cube Notation: http://ruwix.com/the-rubiks-cube/notation/
    def L(self):  self._rotate_face(LEFT, ROT_YZ_CC)
    def Li(self): self._rotate_face(LEFT, ROT_YZ_CW)
    def R(self):  self._rotate_face(RIGHT, ROT_YZ_CW)
    def Ri(self): self._rotate_face(RIGHT, ROT_YZ_CC)
    def U(self):  self._rotate_face(UP, ROT_XZ_CW)
    def Ui(self): self._rotate_face(UP, ROT_XZ_CC)
    def D(self):  self._rotate_face(DOWN, ROT_XZ_CC)
    def Di(self): self._rotate_face(DOWN, ROT_XZ_CW)
    def F(self):  self._rotate_face(FRONT, ROT_XY_CW)
    def Fi(self): self._rotate_face(FRONT, ROT_XY_CC)
    def B(self):  self._rotate_face(BACK, ROT_XY_CC)
    def Bi(self): self._rotate_face(BACK, ROT_XY_CW)
    def M(self):  self._rotate_slice(Y_AXIS + Z_AXIS, ROT_YZ_CC)
    def Mi(self): self._rotate_slice(Y_AXIS + Z_AXIS, ROT_YZ_CW)
    def E(self):  self._rotate_slice(X_AXIS + Z_AXIS, ROT_XZ_CC)
    def Ei(self): self._rotate_slice(X_AXIS + Z_AXIS, ROT_XZ_CW)
    def S(self):  self._rotate_slice(X_AXIS + Y_AXIS, ROT_XY_CW)
    def Si(self): self._rotate_slice(X_AXIS + Y_AXIS, ROT_XY_CC)
    def X(self):  self._rotate_pieces(self.pieces, ROT_YZ_CW)
    def Xi(self): self._rotate_pieces(self.pieces, ROT_YZ_CC)
    def Y(self):  self._rotate_pieces(self.pieces, ROT_XZ_CW)
    def Yi(self): self._rotate_pieces(self.pieces, ROT_XZ_CC)
    def Z(self):  self._rotate_pieces(self.pieces, ROT_XY_CW)
    def Zi(self): self._rotate_pieces(self.pieces, ROT_XY_CC)

    def sequence(self, move_str):
        moves = [getattr(self, name) for name in move_str.split()]
        for move in moves:
            move()

    def colors(self):
        return set(c for piece in self.pieces for c in piece.colors if c is not None)

    def left_color(self): return self[LEFT].colors[0]
    def right_color(self): return self[RIGHT].colors[0]
    def up_color(self): return self[UP].colors[1]
    def down_color(self): return self[DOWN].colors[1]
    def front_color(self): return self[FRONT].colors[2]
    def back_color(self): return self[BACK].colors[2]

    def _color_list(self):
        right = [p.colors[0] for p in sorted(self._face(RIGHT), key=lambda p: (-p.pos.y, -p.pos.z))]
        left  = [p.colors[0] for p in sorted(self._face(LEFT),  key=lambda p: (-p.pos.y, p.pos.z))]
        up    = [p.colors[1] for p in sorted(self._face(UP),    key=lambda p: (p.pos.z, p.pos.x))]
        down  = [p.colors[1] for p in sorted(self._face(DOWN),  key=lambda p: (-p.pos.z, p.pos.x))]
        front = [p.colors[2] for p in sorted(self._face(FRONT), key=lambda p: (-p.pos.y, p.pos.x))]
        back  = [p.colors[2] for p in sorted(self._face(BACK),  key=lambda p: (-p.pos.y, -p.pos.x))]

        return (up + left[0:3] + front[0:3] + right[0:3] + back[0:3]
                   + left[3:6] + front[3:6] + right[3:6] + back[3:6]
                   + left[6:9] + front[6:9] + right[6:9] + back[6:9] + down)

    def flat_str(self):
        return "".join(x for x in str(self) if x not in string.whitespace)

    def __str__(self):
        template = ("    {}{}{}\n"
                    "    {}{}{}\n"
                    "    {}{}{}\n"
                    "{}{}{} {}{}{} {}{}{} {}{}{}\n"
                    "{}{}{} {}{}{} {}{}{} {}{}{}\n"
                    "{}{}{} {}{}{} {}{}{} {}{}{}\n"
                    "    {}{}{}\n"
                    "    {}{}{}\n"
                    "    {}{}{}")

        return "    " + template.format(*self._color_list()).strip()


###################################
## SOLUTION FOR CTF
###################################
seq_str = "U B Ui B Li B R Li B B U F B B Ui B B R R F F B B D D B B D"

cube_str = """   bbw
   ogy
   wgr
yggrygyrogwr
grrwwoyorbyr
boyowrwgobby
   gob
   bbw
   oyw
"""
wcube = Cube(cube_str)
wcube.sequence(seq_str)
print(wcube)
print()

cube_str = """   {EO
   LS)
   ELW
:SMT?P}BLENR
_OSDNEKEGLEY
__URSP_UIUS_
   PLØ
   DTE
   LB_"""
wcube = Cube(cube_str)
wcube.sequence(seq_str)
print(wcube)